In [1]:
from torch.nn import MSELoss
import learn_process as lp
from networks.unet_leiterrl import TurbNetG, UNet
from networks.dummy_network import DummyNet
import visualization.visualize_data as vis
import datetime
import sys
import numpy as np
import tensorboard
from torchsummary import summary
import logging

# autoreload
%load_ext autoreload
%autoreload 2


In [2]:

# # model choice 
# model_choice = sys.argv[1]
# if model_choice == "unet":
#     model = unet_model
# elif model_choice == "fc":
#     model = fc_model
# else:
#     print("model choice not recognized")
#     sys.exit()


# TODO overfit until not possible anymore (dummynet, then unet)
# therefor: try to exclude connections from unet until it overfits properly (loss=0)
# TODO go over input properties (delete some, some from other simulation with no hps?)
# TODO: add 3D data
# TODO : data augmentation, 

# model.to(device)

# print(dataloaders_2D["train"].dataset[0]['x'].shape)

In [88]:
def do_it(inputs, model_name, n_epochs, debugging=False, dataset_name="groundtruth_hps_no_hps/groundtruth_hps_overfit_10", reduce_to_2D_wrong=False):
    # set debugging to true if you dont wont a run folder (with loss history etc and a picture of the result)
    logger = logging.getLogger()
    logger.setLevel(logging.WARNING)  # level: DEBUG, INFO, WARNING, ERROR, CRITICAL

    # parameters of model and training
    loss_fn = MSELoss()
    lr=0.0004 #0.0004

    # model alternatives
    #model = TurbNetG(channelExponent=4, in_channels=4, out_channels=2)
    # TODO too many in channels for unet?
    datasets_2D, dataloaders_2D = lp.init_data(reduce_to_2D=True, reduce_to_2D_wrong=reduce_to_2D_wrong, overfit=True, dataset_name=dataset_name, inputs=inputs) # init data
    
    if model_name == "unet":
        unet_model = UNet(in_channels=len(inputs)+1, out_channels=1).float()
        model = unet_model  # current parameter choice
    elif model_name == "dummy":
        dummy_model = DummyNet(in_channels=len(inputs)+1, out_channels=1).float()
        model = dummy_model

    #print(dataloaders_2D["train"].dataset[0]['x'].shape)
    
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    name_folder=now+"_"+model_name+"_overfit"+dataset_name[-2:]+"_"+"in_"+inputs+"_out_T_"+str(n_epochs)+"epochs"
    #summary(model, (len(inputs)+1, 128, 16))
    
    loss_hist = lp.train_model(model, dataloaders_2D, loss_fn, n_epochs, lr, name_folder=name_folder, debugging=debugging)                   # train model
    error = None
    if not debugging:
        error = vis.plot_exemplary_learned_result(model, dataloaders_2D, name_pic=name_folder+"/plot_y_"+name_folder)    # visualize results, pic in folder runs/current model

    return model, error, loss_hist

In [ ]:
# n_epochs = 10000 #000 #7000   # 60000
# inputs="p"
# model_name="unet"
# #try stride=1 in innerst loop
# # try more layers? (depth) to capture the details?
# 
# model, _, _ = do_it(inputs, model_name, n_epochs, dataset_name="groundtruth_hps_no_hps/groundtruth_hps_overfit_01") #,reduce_to_2D_wrong=True) 

# TODO check out normalization - everywhere excluded?? for unet since unet does batch normalization

In [92]:
# # TODO RUN ALL MODELS THIS EVENING

#test unet with 10 datapoints
n_epochs = 10000     #60000
model_names = ["dummy", "unet"]
inputs_combos = ["p", "xy"] #"yz", "yp", "zp", "yzp", "yzt", "pt", "yztp"]

for model_name in model_names:
    for inputs in inputs_combos:
        print(f"Model: {model_name} with input variables: {inputs}")
        do_it(inputs, model_name, n_epochs, dataset_name="groundtruth_hps_no_hps/groundtruth_hps_overfit_10")

# test xy instead of yz 2D unet
for model_name in model_names:
    for inputs in inputs_combos:
        print(f"Model: {model_name} with input variables: {inputs}")
        do_it(inputs, model_name, n_epochs, reduce_to_2D_wrong=True, dataset_name="groundtruth_hps_no_hps/groundtruth_hps_overfit_10")


Model: dummy with input variables: p


epochs:   2%|▏         | 206/10000 [01:01<52:49,  3.09it/s, loss: 0.2243]  